# Grey Clustering Analysis - Assessment of Natural Disaster Events in the Rio Grande do Sul Area

## 1. Disaster Data

In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
from possibility_function_evaluate import *

In [2]:
BASE_FILE_PATH  = '../Data/Danos_Informados_{year}.xls'
AVAILABLE_YEARS = range(2013, 2024)
disasters_data  = pd.DataFrame()

In [3]:
for year in AVAILABLE_YEARS:
    year_data = pd.read_excel(BASE_FILE_PATH.format(year=year), skiprows=4)
    year_data['Ano'] = year

    disasters_data = pd.concat([disasters_data, year_data])


In [4]:
CATEGORIES_TO_REMOVE = ['15110 - Doenças infecciosas virais', '15120 - Doenças infecciosas bacterianas', '15210 - Infestações de animais', '15230 - Outras Infestações', '22410 - Transporte de produtos perigosos rodoviário', '23110 - Incêndios em plantas e distritos industriais, parques e depósitos', '23120 - Incêndios em aglomerados residenciais', '24100 - Colapso de edificações', '25100 - Transporte passageiros e cargas não perigosas rodoviário']
disasters_data = disasters_data.drop(disasters_data[disasters_data['COBRADE'].isin(CATEGORIES_TO_REMOVE)].index)

## 2. Clustering of Regions Regarding Natural Disaster Affects

### 2.1. Grouping Cities into Intermediate-Regions
The first thing we need to do is group the data that we have (from each municipality) into "intro-regions", as shown in the image bellow. 
![](../Docs/Regiões%20Intermediárias.png)

The data of which intro-region delimitation can be found at IBGE's web-site.

In [5]:
REGIONS_DELIMITATION = pd.read_excel('../Data/regioes_geograficas_composicao_por_municipios_2017_20180911.xls')
REGIONS_DELIMITATION = REGIONS_DELIMITATION[['nome_mun', 'nome_rgint']]
REGIONS_DELIMITATION = REGIONS_DELIMITATION[REGIONS_DELIMITATION['nome_rgint'].isin(['Porto Alegre', 'Pelotas', 'Santa Maria', 'Uruguaiana', 'Ijuí', 'Passo Fundo', 'Caxias do Sul', 'Santa Cruz do Sul - Lajeado'])]

In [6]:
#Because of Interface error between IBGE database and S2ID database, one municipality's name does not match correctly and therefore
#needs to be manually corrected
disasters_data['Município'] = disasters_data['Município'].replace(['Sant`Ana do Livramento'],  "Sant'Ana do Livramento")

In [7]:
disasters_data_by_inter_region = pd.merge(left=disasters_data, right=REGIONS_DELIMITATION, left_on='Município', right_on='nome_mun', how='left')

### 2.2. Group The Data by Intermediate Region and Year

Now it is necessary to group, for each year, the total damage (according to each criteria), for every intermediary region.

In [8]:
disasters_data_by_inter_region_average = disasters_data_by_inter_region.groupby(['nome_rgint', 'Ano'])[['DH_Mortos', 'DH_Feridos', 'DH_Enfermos','DH_Desabrigados', 'DH_Desalojados', 'DH_Desaparecidos', 'DH_Outros Afetados', 'DM_Unidades Habitacionais Danificadas', 'DM_Unidades Habitacionais Destruídas', 'DM_Unidades Habitacionais Valor']].sum().groupby(['nome_rgint']).mean()
disasters_data_by_inter_region_average

,DH_Mortos,DH_Feridos,DH_Enfermos,DH_Desabrigados,DH_Desalojados,DH_Desaparecidos,DH_Outros Afetados,DM_Unidades Habitacionais Danificadas,DM_Unidades Habitacionais Destruídas,DM_Unidades Habitacionais Valor
nome_rgint,,,,,,,,,,
Caxias do Sul,0.272727,9.000000,0.000000,73.363636,465.545455,0.000000,34657.090909,624.454545,18.272727,1.585272e+07
Ijuí,0.363636,4.545455,0.272727,22.000000,82.181818,0.000000,124601.545455,1449.636364,7.545455,2.469623e+06
Passo Fundo,0.363636,8.818182,6.090909,490.454545,543.727273,0.000000,203136.090909,3309.000000,22.818182,4.829483e+06
Pelotas,0.090909,0.000000,0.090909,96.454545,489.000000,0.000000,159559.454545,3090.000000,3.545455,2.832020e+06
Porto Alegre,0.818182,312.181818,90.090909,1027.727273,9783.909091,0.272727,138664.545455,8785.363636,50.272727,2.736912e+07
Santa Cruz do Sul - Lajeado,0.454545,2.818182,0.909091,459.090909,1962.909091,83.363636,103256.636364,5045.272727,33.545455,7.341957e+07
Santa Maria,0.000000,3.363636,2.454545,72.272727,1377.545455,0.090909,175817.272727,2108.636364,7.727273,4.076717e+06
Uruguaiana,0.000000,5.181818,1.090909,315.363636,2480.272727,4.090909,94724.727273,1024.090909,26.090909,3.663741e+06


### 2.3. Possibility Functions

Calculate the possibility functions for each criteria. The possibility functions are represented as $[x_1, x_2, x_3, x_4]$

In [9]:
data_min = disasters_data_by_inter_region_average.min()
data_max = disasters_data_by_inter_region_average.max()
data_mean = disasters_data_by_inter_region_average.mean()

lower_measure_possibility_function = {}
lower_measure_possibility_function['x3'] = (data_min + data_mean)/2
lower_measure_possibility_function['x4'] = data_mean

moderate_measure_possibility_function = {}
moderate_measure_possibility_function['x1'] = (data_min + data_mean)/2
moderate_measure_possibility_function['x2'] = data_mean
moderate_measure_possibility_function['x4'] = (data_max + data_mean)/2

upper_measure_possibility_function = {}
upper_measure_possibility_function['x1'] = data_mean
upper_measure_possibility_function['x2'] = (data_max + data_mean)/2

In [10]:
lower_measure_possibility_function

{'x3': DH_Mortos                                1.477273e-01
 DH_Feridos                               2.161932e+01
 DH_Enfermos                              6.312500e+00
 DH_Desabrigados                          1.707955e+02
 DH_Desalojados                           1.115159e+03
 DH_Desaparecidos                         5.488636e+00
 DH_Outros Afetados                       8.197963e+04
 DM_Unidades Habitacionais Danificadas    1.902006e+03
 DM_Unidades Habitacionais Destruídas     1.238636e+01
 DM_Unidades Habitacionais Valor          9.641873e+06
 dtype: float64,
 'x4': DH_Mortos                                2.954545e-01
 DH_Feridos                               4.323864e+01
 DH_Enfermos                              1.262500e+01
 DH_Desabrigados                          3.195909e+02
 DH_Desalojados                           2.148136e+03
 DH_Desaparecidos                         1.097727e+01
 DH_Outros Afetados                       1.293022e+05
 DM_Unidades Habitacionais Danificad

### 2.4. Criteria Weights
Since all the criteria are of equal importance, all weights shall be the same:

In [11]:
n_criteria = disasters_data_by_inter_region_average.shape[1]
criteria_weights = np.ones((n_criteria,1)) / n_criteria

criteria_weights_table = pd.DataFrame(criteria_weights, index=disasters_data_by_inter_region_average.columns, columns=['Criteria Weight'])
criteria_weights_table

,Criteria Weight
DH_Mortos,0.1
DH_Feridos,0.1
DH_Enfermos,0.1
DH_Desabrigados,0.1
DH_Desalojados,0.1
DH_Desaparecidos,0.1
DH_Outros Afetados,0.1
DM_Unidades Habitacionais Danificadas,0.1
DM_Unidades Habitacionais Destruídas,0.1
DM_Unidades Habitacionais Valor,0.1


### 2.5 Possibility Calculations

In [12]:
region_evaluation = {}
for intermediate_region in disasters_data_by_inter_region_average.index.values:
    # Calculate the possibility values for lower function for all 
    f_xs = evaluate_lower_possibility(disasters_data_by_inter_region_average.loc[intermediate_region, :], lower_measure_possibility_function)
    # Multiply the corresponding weight and calculate resulting sigma
    lower_sigma = np.sum(f_xs * criteria_weights)

    # Calculate the possibility values for moderate function for all 
    f_xs = evaluate_moderate_possibility(disasters_data_by_inter_region_average.loc[intermediate_region, :], moderate_measure_possibility_function)
    # Multiply the corresponding weight and calculate resulting sigma
    moderate_sigma = np.sum(f_xs * criteria_weights)

    # Calculate the possibility values for upper function for all 
    f_xs = evaluate_upper_possibility(disasters_data_by_inter_region_average.loc[intermediate_region, :], upper_measure_possibility_function)
    # Multiply the corresponding weight and calculate resulting sigma
    severe_sigma = np.sum(f_xs * criteria_weights)

    region_evaluation[intermediate_region] = {'Severe Impact': severe_sigma, 'Moderate Impact':moderate_sigma, 'Low Impact':lower_sigma}


In [13]:
pd.DataFrame(region_evaluation).transpose()[['Low Impact', 'Moderate Impact', 'Severe Impact']].style.highlight_max(color='blue', axis=1)

,Low Impact,Moderate Impact,Severe Impact
Caxias do Sul,0.762208,0.237792,0.000000
Ijuí,0.809933,0.163980,0.026087
Passo Fundo,0.500000,0.310083,0.189917
Pelotas,0.807010,0.111030,0.081960
Porto Alegre,0.100000,0.137346,0.762654
Santa Cruz do Sul - Lajeado,0.272970,0.275378,0.451652
Santa Maria,0.858425,0.041575,0.100000
Uruguaiana,0.675909,0.281902,0.042189


## 3. Clustering of Natural Disaster Season (Year)

In [14]:
disasters_data_by_year = disasters_data[['Ano', 'DH_Mortos', 'DH_Feridos', 'DH_Enfermos','DH_Desabrigados', 'DH_Desalojados', 'DH_Desaparecidos', 'DH_Outros Afetados', 'DM_Unidades Habitacionais Danificadas', 'DM_Unidades Habitacionais Destruídas', 'DM_Unidades Habitacionais Valor']]

In [15]:
disasters_data_by_year = disasters_data_by_year.groupby(['Ano']).sum()

In [19]:
disasters_data_by_year

,DH_Mortos,DH_Feridos,DH_Enfermos,DH_Desabrigados,DH_Desalojados,DH_Desaparecidos,DH_Outros Afetados,DM_Unidades Habitacionais Danificadas,DM_Unidades Habitacionais Destruídas,DM_Unidades Habitacionais Valor
Ano,,,,,,,,,,
2013,5,115,812,12380,46429,1,531830,24734,360,8.715458e+07
2014,1,71,29,2516,15521,0,264969,16362,229,3.035127e+07
2015,0,24,0,2908,23701,46,351192,39584,147,9.570269e+07
2016,2,346,8,1440,18321,916,243990,16636,223,7.186598e+08
2017,5,133,1,638,18171,0,907599,30764,442,7.680782e+07
2018,4,44,4,243,2111,0,353810,9449,72,2.521029e+07
2019,1,6,1,339,3579,0,501589,15539,9,2.827599e+07
2020,0,8,187,17,70,0,1252656,1187,4,1.892718e+06
2021,0,27,0,12,397,0,561831,38537,28,1.563735e+08


In [16]:
data_min = disasters_data_by_year.min()
data_max = disasters_data_by_year.max()
data_mean = disasters_data_by_year.mean()

lower_measure_possibility_function = {}
lower_measure_possibility_function['x3'] = (data_min + data_mean)/2
lower_measure_possibility_function['x4'] = data_mean

moderate_measure_possibility_function = {}
moderate_measure_possibility_function['x1'] = (data_min + data_mean)/2
moderate_measure_possibility_function['x2'] = data_mean
moderate_measure_possibility_function['x4'] = (data_max + data_mean)/2

upper_measure_possibility_function = {}
upper_measure_possibility_function['x1'] = data_mean
upper_measure_possibility_function['x2'] = (data_max + data_mean)/2

In [17]:
season_evaluation = {}
for year in disasters_data_by_year.index.values:
    print(year)
    # Calculate the possibility values for lower function for all 
    f_xs = evaluate_lower_possibility(disasters_data_by_year.loc[year, :], lower_measure_possibility_function)
    # Multiply the corresponding weight and calculate resulting sigma
    lower_sigma = np.sum(f_xs * criteria_weights)

    # Calculate the possibility values for moderate function for all 
    f_xs = evaluate_moderate_possibility(disasters_data_by_year.loc[year, :], moderate_measure_possibility_function)
    # Multiply the corresponding weight and calculate resulting sigma
    moderate_sigma = np.sum(f_xs * criteria_weights)

    # Calculate the possibility values for upper function for all 
    f_xs = evaluate_upper_possibility(disasters_data_by_year.loc[year, :], upper_measure_possibility_function)
    # Multiply the corresponding weight and calculate resulting sigma
    severe_sigma = np.sum(f_xs * criteria_weights)

    season_evaluation[year] = {'Severe Impact': severe_sigma, 'Moderate Impact':moderate_sigma, 'Low Impact':lower_sigma}


2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023


In [18]:
pd.DataFrame(season_evaluation).transpose()[['Low Impact', 'Moderate Impact', 'Severe Impact']].style.highlight_max(color='blue', axis=1)

,Low Impact,Moderate Impact,Severe Impact
2013,0.377213,0.122787,0.500000
2014,0.697489,0.259024,0.043487
2015,0.581288,0.283263,0.135448
2016,0.391120,0.364084,0.244796
2017,0.519112,0.239955,0.240933
2018,0.900000,0.029412,0.070588
2019,0.981630,0.018370,0.000000
2020,0.800000,0.164736,0.035264
2021,0.800000,0.104048,0.095952
2022,0.800000,0.135757,0.064243
